### Data 620 - Project 1
### Bikram Barua and Leticia Salazar
### February 19, 2023

#### Task: 

For your first project, you are asked to:
1. Identify and load a network dataset that has some categorical information available for each node.
2. For each of the nodes in the dataset, calculate degree centrality and eigenvector centrality.
3. Compare your centrality measures across your categorical groups.

For example, using the dataset from a study of romantic relationships among high school students in Columbus, Ohio [http://researchnews.osu.edu/archive/chains.htm], you might want to determine if there are any differences in your calculated centrality measures between the two sexes.  You might use a t-test or another statistical measure to support your findings.

#### Data source:

The dataset selected for project 1 was obtained from [Network Repository](https://networkrepository.com/socfb-Cornell5.php) consisting of people (nodes) and their friendship ties (edges) on Facebook. The data is available in a matrix market, typically a sparse format used to represent a matrix (.mtx file). With these types of files, the first line contains a header with information about the matrix, dimensions, and symmetry of the matrix. The following lines are non-zero entries of the matrix in row-major order, with each following line containing the row index, column index, and a value of a single non-zero entry.

Below are some specs from this dataset provided by the repository:

* Nodes: 18.7K
* Edges: 790.8K
* Density: 0.00454239
* Maximum degree: 3.2K
* Minimum degree: 1
* Average degree: 84
* Assortativity: 0.01805
* Number of triangles: 18.3M
* Average number of triangles: 982
* Maximum number of triangles: 77.2K
* Average clustering coefficient: 0.219051
* Fraction of closed triangles: 0.135615
* Maximum k-core: 85
* Lower bound of Maximum Clique: 13

#### Project Objectives: 

We will be focusing to answer the following while we explore, analyze and create visualizations for our data.

* Which centrality measure would be the most relevant in the dataset?
* What would these centrality measures help us to predict?
* What statistical tests can we apply to identify the best fit centrality measure?

#### Load libraries:

Below are the libraries used

In [1]:
# data manipulation
import pandas as pd
import numpy as np
import scipy.io #for mtx files
from tqdm import tqdm #Instantly make your loops show a smart progress meter 

# data visualization
import networkx as nx
import matplotlib.pyplot as plt

#### Load the data:
Data exploration of the data is performed below to view the size of the datasets we will be working with. As mentioned, the original file had to be converted to a csv file for us to work with the dataset.

In [2]:
# Don't mind this I am working on reading the mtx file from github
#df = scipy.io.mmread('https://raw.githubusercontent.com/letisalba/Data-620/master/Week-4/Cornell5.mtx')
#df = mmread(Cornell5.mtx)

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/letisalba/Data-620/master/Week-4/fb-cornell5.csv', header = None)
# rename column headers
df.columns = ["People", "Friendships"]
print(df.head(10))

   People  Friendships
0     184            1
1     344            1
2     635            1
3     867            1
4    1296            1
5    1358            1
6    1425            1
7    1501            1
8    1505            1
9    1845            1


The dataset is comprised of 790775 variables 

In [3]:
# dimension of data
len(df)

790776

#### Create NetworkX Graph

In [5]:
# start networkX 
G = nx.Graph()
G.add_nodes_from(df)

#G = nx.from_pandas_edgelist(df, source='People', target='Friendships')
#fig, ax = plt.subplots(figsize=(10,8))
#nx.draw_networkx(G, ax=ax)

In [6]:
for i, elrow in df.iterrows():
    G.add_node(elrow[0])
    G.add_node(elrow[1])

    if G.has_edge(elrow[0], elrow[1]):
            # edge already exists, increase weight by one
            G[elrow[0]][elrow[1]]['weight'] += 1
    else:
            # add new edge with weight 1
            G.add_edge(elrow[0], elrow[1], weight = 1)


G_nodes = G.number_of_nodes()
G_edges = G.number_of_edges()
print("Nodes = ", G_nodes, " Edges = ",G_edges)

Nodes =  18662  Edges =  790776


In [7]:
nx.write_gexf(G, 'project1-network.gexf')

In [8]:
# Preview first 5 edges

list(G.edges(data=True))[0:5]

[(184, 1, {'weight': 1}),
 (184, 5, {'weight': 1}),
 (184, 136, {'weight': 1}),
 (184, 226, {'weight': 1}),
 (184, 519, {'weight': 1})]

In [9]:
# Preview first 10 nodes
list(G.nodes(data=True))[0:10]

[('People', {}),
 ('Friendships', {}),
 (184, {}),
 (1, {}),
 (344, {}),
 (635, {}),
 (867, {}),
 (1296, {}),
 (1358, {}),
 (1425, {})]

In [12]:
pos = nx.spring_layout(df)
betCent = nx.betweenness_centrality(df, normalized=True, endpoints=True)
node_color = [20000.0 * df.degree(v) for v in df]
node_size =  [v * 10000 for v in betCent.values()]
plt.figure(figsize=(20,20))
nx.draw_networkx(df, pos=pos, with_labels=False,
                 node_color=node_color,
                 node_size=node_size )
plt.axis('off')
sorted(betCent, key=betCent.get, reverse=True)[:5]

KeyError: 184

In [21]:
# Calculate degree centrality
degree = nx.degree(G)
df_2 = pd.DataFrame(degree, columns = ['People','Degree_Cent'])
df_2 = pd.merge(df_2, df, on='People', how='inner')
df2 = df_2.sort_values('Degree_Cent', ascending = False)
print(df2.head())
print(df2.tail())

        People  Degree_Cent  Friendships
105721   17027         3156          337
107116   17027         3156         8614
107136   17027         3156         8723
107135   17027         3156         8712
107134   17027         3156         8711
        People  Degree_Cent  Friendships
757617    8916            1          789
790115   10051            1         5082
790537    9224            1         7670
788694    3833            1         3036
790775   18575            1        16538


In [22]:
# Calculate eigenvector centrality
eigen = nx.eigenvector_centrality(G)
df_3 = pd.DataFrame.from_dict(eigen, orient='index').reset_index()
df_3.columns = ['People', 'Eigenvec_Cent']
df_3 = pd.merge(df_3, df, on='People', how='inner')
df_3 = df_3.sort_values('Eigenvec_Cent', ascending = False)
df_3.head()

,People,Eigenvec_Cent,Friendships
789851,17058,3.558682e-58,4320
790752,15288,3.558682e-58,13467
790725,13542,3.558682e-58,11936
786310,3671,3.558682e-58,2127
789174,8416,3.558682e-58,3473


In [ ]:
df_3.tail()

In [23]:
# Calculate closeness centrality
df_4 = nx.closeness_centrality(G)

#### References:

* Rossi, R. A., & Ahmed, N. K. (2015). The Network Data Repository with Interactive Graph Analytics and Visualization. Network Data Repository. Retrieved February 16, 2023, from https://networkrepository.com 
* AOMAR, A. A. I. T. (2020, August 2). Notes on graph theory - centrality measures. Medium. Retrieved February 17, 2023, from https://towardsdatascience.com/notes-on-graph-theory-centrality-measurements-e37d2e49550a 
* Disney, A. (2022, October 27). Social network analysis: Understanding centrality measures. Cambridge Intelligence. Retrieved February 18, 2023, from https://cambridge-intelligence.com/keylines-faqs-social-network-analysis/ 
* ArijitGayen@ArijitGayen. (2019, November 21). Network centrality measures in a graph using Networkx: Python. GeeksforGeeks. Retrieved February 18, 2023, from https://www.geeksforgeeks.org/network-centrality-measures-in-a-graph-using-networkx-python/ 